# 一、简介

在这篇教程中,我们将一起实现一个简单的 Retrieval-Augmented Generation（RAG）检索增强生成应用,检索增强生成顾名思义,就是利用检索来增强大模型的生成结果.

具体而言, RAG 主要是在这样的场景下被需要的:想象一下,当你有一个冷门的知识需要理解,但大模型没有基于这个知识训练过,或者说你想把这个知识全部输入大模型进行问答,但是大模型的上下文没有那么长;那么,我们需要一个好的方法让大模型可以基于我们新的知识进行对话,这就是 RAG的意义所在.

检索增强生成 (RAG) 通过向量数据库检索的方式来获取我们问题预期想要回答涉及的知识,然后结合这个知识让大模型基于知识生成最后的问答结果,满足了我们对提问的真实性需求.

在理解 RAG 之前,我们还需要理解什么是 Embedding.在机器学习和自然语言处理（NLP）中，Embedding 是一种将非结构化数据,如单词、句子或者整个文档，转化为实数向量的技术。这些实数向量可以被计算机更好地理解和处理.我们可以把一个词(token)表示成有限维度空间中的表示,如我们可以把苹果映射成 (5,5) ,把梨子映射成 (4,5),把芯片映射到 (1,2). 在这里,坐标的相近表示梨子和苹果的语义有很大的重复成分,而芯片与苹果的距离,自然比梨子与苹果的距离要远. 此时这些数字坐标映射就可以理解为简单的 Embedding, 我们可以通过 Embedding 模型,将一个词很方便映射到对应的实数多维表示空间,这个映射关系是提前训练得到的,越准确的 Embedding 模型可以让我们越好的区别不同语义特征的差异性,这也就对 RAG 的准确检索带来了更大的好处.

在搭建 RAG 系统时，我们往往可以通过使用 Embedding 模型来构建词向量，我们可以选择使用各个公司的在线 Embedding API,也可以使用本地嵌入模型将数据构建为词向量;由于我们通常是对文档操作,这里的向量化是对文档块(chunk)进行;我们可以将一个文档分成多个段落,每个段落分别进行 Embedding 操作,得到结果后存储到对应的数据库中保存,以便后续的检索即可.

而对于数据库,在 RAG 中,我们通常使用的也就是 Embedding 相关的数据库 - 向量数据库. 向量数据库是一种基于向量空间模型的数据库系统,它能够利用向量运算进行数据检索的高效处理,常见的向量数据库包括 Faiss、Annoy、Milvus 等等。这些向量数据库通常与 LLM 结合使用,以提高数据检索和处理的效率。

至此,在理解了 Embedding 之后,我们就理解了 RAG 中的一大核心要素.那么,我们将如何构建一个 RAG 系统?

具体而言, RAG 有很多的实现方式,在这里我们使用最简单的实现方法,他遵循最传统的规则,包括索引创建（Indexing）、检索（Retrieval）和生成（Generation）,总的来说包括以下三个关键步骤：
- 语料库被划分成一个个分散的块(chunk)，然后使用embedding模型构建向量索引,并存储到向量数据库.
- RAG 根据 query (当前提问)与索引块（Indexed Chunk）的向量相似度识别并对块进行检索。
- 模型根据检索块（Retrieved Chunk）中获取的上下文信息生成答案。

RAG 也可以被简单的分成几大模块:  
- 向量化模块，用来将文档片段向量化。
- 文档加载和切分的模块工具，用来加载文档并切分成文档片段。
- 向量数据库模块,用于将向量化后的文档存储到数据库中.
- 检索模块，根据 Query （问题）检索相关的文档片段。
- 大模型模块，结合 Query 及检索出来的文档回答用户的问题。

我们可以用一张图简单理解 RAG 系统做了哪些事情:

![](https://bce.bdstatic.com/community/uploads/community_30492e9.png)

由图可知,我们通过基于提问检索出的知识块,和提问一起拼接输入到大模型问答后,让大模型的回答更加接近我们的提问预期,可靠性大幅度增加.但这也并非 RAG 技术的终点,我们可以通过更多额外方式增强 RAG 的效果,譬如:

![](https://bce.bdstatic.com/community/uploads/community_3859ff4.png)

以最简单的"重排技术"为例,我们可以通过检索后重新排布检索的结果,从而提高提高打算使用的检索块与提问的关联度,最终提高问答的生成质量。在 RAG 架构下，引入重排步骤可以有效改进召回效果，提升LLM（大语言模型）生成答案的质量。我们可以通过一张图简单理解这一过程:

![](https://file.techbeat.net/2024/2024-01-09/upload/png/5005bef2f31b404577d638e270410f14.png)

总之,我们可以利用 RAG 技术提高大模型问答最后的生成水平,在强事实要求与上下文不足的情况下仅仅依靠 RAG 就能实现满足预期的效果. 接下来, 让我们从初级 RAG 开始, 一步步探索检索增强生成的应用之路.




# 二、实现 RAG 模块

在上一篇教程中,我们已经学会了如何应用 IPEX-LLM 加速推理,也学会了如何用 Gradio 与 streamlit 实现一个简单的前端演示界面,所以,在这篇教程中,我们能够快速学会如何搭建一个基于 IPEX-LLM 的 RAG 系统.

我们将使用 LlamaIndex 进行 RAG 系统的搭建演示, LlamaIndex是一个AI框架，用于简化将私有数据与公共数据集成到大型语言模型（LLM）中的应用程序中。它提供了数据 ingestion、 indexing 和查询的工具，使其成为生成式AI需求的可靠解决方案。

LlamaIndex主要包括以下几个组件:
- 数据连接器：帮助连接现有数据源和数据格式（如API、PDF等），并将这些数据转换为LlamaIndex可用的格式。
- 数据索引：帮助结构化数据以适应不同的用例。加载了来自不同数据源的数据后，如何将它们分割、定义关系和组织，以便无论您想要解决的问题（问答、摘要等），都可以使用索引来检索相关信息。
- 查询接口：是输入查询并从LLM中获取知识增强输出的接口。

看了那么多有关 LlamaIndex 的介绍,我们还是不能学会如何结合它与 IPEX-LLM 实现一个简单的 RAG 系统,所以,让我们直接进入到代码的阅读环节.

我们可以来看一个简单的 LlamaIndex 示例,它直观展示了如何构建一个 RAG 体系:

假设你有如下的文件组织:
```
├── starter.py
└── data
    └── paul_graham_essay.txt
```

核心代码为:
```python
# 导入需要的模块和类
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

# 1. 使用 SimpleDirectoryReader 加载数据
# SimpleDirectoryReader 是一个简单的目录读取器，能从指定目录中读取所有文件的数据
documents = SimpleDirectoryReader("data").load_data()

# 2. 设置嵌入模型为 bge-base
# HuggingFaceEmbedding 是一个嵌入模型类，用于将文本转换为向量表示
# 这里我们使用的是 "BAAI/bge-base-en-v1.5" 模型
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# 3. 使用 Ollama 快速接入大语言模型
# Ollama 是一个模型的快速调用框架
# 这里我们指定使用 "llama3" 模型，并设置请求超时时间为 360 秒
Settings.llm = Ollama(model="llama3", request_timeout=360.0)

# 4. 创建一个向量存储索引
# VectorStoreIndex 是一个用于存储和查询向量的索引类
# from_documents 方法是从文档数据创建索引
index = VectorStoreIndex.from_documents(documents)

# 5. 将索引转换为查询引擎
# as_query_engine 方法将现有的向量存储索引转换为一个查询引擎
# 查询引擎用来对存储的数据进行语义查询
query_engine = index.as_query_engine()

# 6. 使用查询引擎进行查询
# query 方法接受一个查询字符串，并返回一个响应对象
# 这里我们查询 "作者小时候做了什么？"
response = query_engine.query("What did the author do growing up?")

# 7. 打印查询结果
# 打印从查询引擎返回的响应
print(response)
```

从代码中,我们可以很容易看到 RAG 系统构建过程,首先我们需要一个读取器来获得某个目录的对应数据,接着需要对这个数据进行 embedding 化即创建索引,把他转为向量表示,最后就可以用设定好的大模型,结合 query 与进行检索增强生成的对话.

接下来,我们将基于 Llamaindex 正式构建一个简易的 RAG 系统,首先我们需要下载中文 Embedding 模型并安装 RAG 系统所需的全部依赖,在这里我们将使用 pdf 文件作为示范.

你需要在终端中执行下列命令安装所有依赖:
```bash
cd /mnt/workspace
conda activate ipex
pip install PyMuPDF llama-index-vector-stores-chroma llama-index-readers-file llama-index-embeddings-huggingface llama-index
```

In [ ]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
# 第一个参数表示下载模型的型号，第二个参数是下载后存放的缓存地址，第三个表示版本号，默认 master
model_dir = snapshot_download('AI-ModelScope/bge-small-zh-v1.5', cache_dir='qwen2chat_src', revision='master')

在成功进行推理前,我们需要准备 RAG 使用的 pdf 文件进行向量化处理,你可以按照如下文件组织存放对应的 pdf 文件,值得注意的是,在这里推荐只使用15页内的 pdf 作为快速测试,先跑通后再考虑加大 pdf 的页数;或者你可以考虑提前构建好向量数据库,此时直接使用 LlamaIndex 再次加载即可使用.

```
├── run_rag.py
└── data
    └── llama2tiny.pdf
```

存放完 pdf 后,你需要修改下列 run_rag.py 代码块文件中的 `Config` 配置,将 question 修改成你想要对材料的提问,将 data_path 设定为 pdf 的文件地址,随后可以运行代码块创建出带运行的 RAG python 工程文件,紧接着按照下列方式即可启动:

```python
cd /mnt/workspace
conda activate ipex
python3 run_rag.py
```

In [5]:
%%writefile /mnt/workspace/run_rag.py
# 设置OpenMP线程数为8
import os
import time
os.environ["OMP_NUM_THREADS"] = "8"

import torch
from typing import Any, List, Optional


# 从llama_index库导入HuggingFaceEmbedding类，用于将文本转换为向量表示
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# 从llama_index库导入ChromaVectorStore类，用于高效存储和检索向量数据
from llama_index.vector_stores.chroma import ChromaVectorStore
# 从llama_index库导入PyMuPDFReader类，用于读取和解析PDF文件内容
from llama_index.readers.file import PyMuPDFReader
# 从llama_index库导入NodeWithScore和TextNode类
# NodeWithScore: 表示带有相关性分数的节点，用于排序检索结果
# TextNode: 表示文本块，是索引和检索的基本单位。节点存储文本内容及其元数据，便于构建知识图谱和语义搜索
from llama_index.core.schema import NodeWithScore, TextNode
# 从llama_index库导入RetrieverQueryEngine类，用于协调检索器和响应生成，执行端到端的问答过程
from llama_index.core.query_engine import RetrieverQueryEngine
# 从llama_index库导入QueryBundle类，用于封装查询相关的信息，如查询文本、过滤器等
from llama_index.core import QueryBundle
# 从llama_index库导入BaseRetriever类，这是所有检索器的基类，定义了检索接口
from llama_index.core.retrievers import BaseRetriever
# 从llama_index库导入SentenceSplitter类，用于将长文本分割成句子或语义完整的文本块，便于索引和检索
from llama_index.core.node_parser import SentenceSplitter
# 从llama_index库导入VectorStoreQuery类，用于构造向量存储的查询，支持语义相似度搜索
from llama_index.core.vector_stores import VectorStoreQuery
# 向量数据库
import chromadb
from ipex_llm.llamaindex.llms import IpexLLM

class Config:
    """配置类,存储所有需要的参数"""
    model_path = "qwen2chat_int4"
    tokenizer_path = "qwen2chat_int4"
    question = "How does Llama 2 perform compared to other open-source models?"
    data_path = "./data/llamatiny.pdf"
    persist_dir = "./chroma_db"
    embedding_model_path = "qwen2chat_src/AI-ModelScope/bge-small-zh-v1___5"
    max_new_tokens = 64

def load_vector_database(persist_dir: str) -> ChromaVectorStore:
    """
    加载或创建向量数据库
    
    Args:
        persist_dir (str): 持久化目录路径
    
    Returns:
        ChromaVectorStore: 向量存储对象
    """
    # 检查持久化目录是否存在
    if os.path.exists(persist_dir):
        print(f"正在加载现有的向量数据库: {persist_dir}")
        chroma_client = chromadb.PersistentClient(path=persist_dir)
        chroma_collection = chroma_client.get_collection("llama2_paper")
    else:
        print(f"创建新的向量数据库: {persist_dir}")
        chroma_client = chromadb.PersistentClient(path=persist_dir)
        chroma_collection = chroma_client.create_collection("llama2_paper")
    print(f"Vector store loaded with {chroma_collection.count()} documents")
    return ChromaVectorStore(chroma_collection=chroma_collection)

def load_data(data_path: str) -> List[TextNode]:
    """
    加载并处理PDF数据
    
    Args:
        data_path (str): PDF文件路径
    
    Returns:
        List[TextNode]: 处理后的文本节点列表
    """
    loader = PyMuPDFReader()
    documents = loader.load(file_path=data_path)

    text_parser = SentenceSplitter(chunk_size=384)
    text_chunks = []
    doc_idxs = []
    for doc_idx, doc in enumerate(documents):
        cur_text_chunks = text_parser.split_text(doc.text)
        text_chunks.extend(cur_text_chunks)
        doc_idxs.extend([doc_idx] * len(cur_text_chunks))

    nodes = []
    for idx, text_chunk in enumerate(text_chunks):
        node = TextNode(text=text_chunk)
        src_doc = documents[doc_idxs[idx]]
        node.metadata = src_doc.metadata
        nodes.append(node)
    return nodes

class VectorDBRetriever(BaseRetriever):
    """向量数据库检索器"""

    def __init__(
        self,
        vector_store: ChromaVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """
        检索相关文档
        
        Args:
            query_bundle (QueryBundle): 查询包
        
        Returns:
            List[NodeWithScore]: 检索到的文档节点及其相关性得分
        """
        query_embedding = self._embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = self._vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))
        print(f"Retrieved {len(nodes_with_scores)} nodes with scores")
        return nodes_with_scores

def completion_to_prompt(completion: str) -> str:
    """
    将完成转换为提示格式
    
    Args:
        completion (str): 完成的文本
    
    Returns:
        str: 格式化后的提示
    """
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

def messages_to_prompt(messages: List[dict]) -> str:
    """
    将消息列表转换为提示格式
    
    Args:
        messages (List[dict]): 消息列表
    
    Returns:
        str: 格式化后的提示
    """
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    prompt = prompt + "<|assistant|>\n"

    return prompt

def setup_llm(config: Config) -> IpexLLM:
    """
    设置语言模型
    
    Args:
        config (Config): 配置对象
    
    Returns:
        IpexLLM: 配置好的语言模型
    """
    return IpexLLM.from_model_id_low_bit(
        model_name=config.model_path,
        tokenizer_name=config.tokenizer_path,
        context_window=384,
        max_new_tokens=config.max_new_tokens,
        generate_kwargs={"temperature": 0.7, "do_sample": False},
        model_kwargs={},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="cpu",
    )
def main():
    """主函数"""
    config = Config()
    
    # 设置嵌入模型
    embed_model = HuggingFaceEmbedding(model_name=config.embedding_model_path)
    
    # 设置语言模型
    llm = setup_llm(config)
    
    # 加载向量数据库
    vector_store = load_vector_database(persist_dir=config.persist_dir)
    
    # 加载和处理数据
    nodes = load_data(data_path=config.data_path)
    for node in nodes:
        node_embedding = embed_model.get_text_embedding(
            node.get_content(metadata_mode="all")
        )
        node.embedding = node_embedding
    
    # 将 node 添加到向量存储
    vector_store.add(nodes)
    
    # 设置查询
    query_str = config.question
    query_embedding = embed_model.get_query_embedding(query_str)
    
    # 执行向量存储检索
    print("开始执行向量存储检索")
    query_mode = "default"
    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)

    # 处理查询结果
    print("开始处理检索结果")
    nodes_with_scores = []
    for index, node in enumerate(query_result.nodes):
        score: Optional[float] = None
        if query_result.similarities is not None:
            score = query_result.similarities[index]
        nodes_with_scores.append(NodeWithScore(node=node, score=score))
    
    # 设置检索器
    retriever = VectorDBRetriever(
        vector_store, embed_model, query_mode="default", similarity_top_k=1
    )
    
    print(f"Query engine created with retriever: {type(retriever).__name__}")
    print(f"Query string length: {len(query_str)}")
    print(f"Query string: {query_str}")
    
    # 创建查询引擎
    print("准备与llm对话")
    query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

    # 执行查询
    print("开始RAG最后生成")
    start_time = time.time()
    response = query_engine.query(query_str)

    # 打印结果
    print("------------RESPONSE GENERATION---------------------")
    print(str(response))
    print(f"inference time: {time.time()-start_time}")

if __name__ == "__main__":
    main()

Overwriting /mnt/workspace/run_rag.py


当你能够看到正常的返回值,则证明你已经顺利实现了简单的 RAG 问答系统,接下来你可以尝试修改这个系统,或是用其他的 RAG 实现方法,来增强当前的问答体验.

祝你好运!

# Reference 

- [ipex-llm LlamaIndex Examples](https://github.com/intel-analytics/ipex-llm/tree/main/python/llm/example/CPU/LlamaIndex)
- [Retrieval-Augmented Generation for AI-Generated Content: A Survey](https://arxiv.org/abs/2402.19473)
- [基于 Milvus + LlamaIndex 实现高级 RAG](https://segmentfault.com/a/1190000044902164)
- [llamaindex](https://docs.llamaindex.ai/en/stable/)
- [llm-universe](https://github.com/datawhalechina/llm-universe/tree/main)
- [tiny-universe](https://github.com/datawhalechina/tiny-universe/tree/main)